In [2]:
import sys
sys.path.append('../src/mane/prototype/')
import numpy as np
import graph as g
import pickle as p

from sklearn.preprocessing import normalize, scale
from sklearn.metrics import f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [33]:
def lg(exp_id, graph_name, index=[0], norm=False, split=0.5, use_bias=False,
       max_iter=100, C=1e9, ic=200, test_with_training_data=True):
    weightfile = '../src/mane/prototype/embeddings/' + exp_id + '.weights'
    graphfile = '../src/mane/data/' + graph_name
    with open(weightfile, 'rb') as f:
        w = p.load(f)
    graph = g.graph_from_pickle(graphfile+'.graph', graphfile+'.community')
    emb = None
    if index is None:
        emb = w
    else:
        for i in index:
            if emb is None:
                emb = w[i]
            else:
                emb += w[i]
        emb /= len(index)
    if use_bias:
        emb[:,-1] = w[2].reshape((-1,))
    if norm:
        emb = normalize(emb)
    xids, y_train = graph.gen_training_community(split)
    X = [emb[i] for i in xids]
    predictor = LogisticRegression(C=C, max_iter=max_iter, 
                                   n_jobs=-1, intercept_scaling=ic).fit(X, y_train)
    if test_with_training_data:
        eval_list = graph.nodes()
    else:
        eval_list = [i for i in graph.nodes() if i not in xids]
    y_true = [graph._communities[i] for i in eval_list]
    y_pred = [predictor.predict(emb[i].reshape(1,-1))[0] for i in eval_list]
    print('Experiment ', exp_id, ' ', graph_name, ' ', str(index))
    print('f1_macro: ', f1_score(y_true, y_pred, average='macro'))
    print('f1_micro: ', f1_score(y_true, y_pred, average='micro'))

In [5]:
lg('BC3041', 'blogcatalog3', [0,1], True)

Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.280467818713
f1_micro:  0.321470131885


In [34]:
for _ in range(10):
    lg('BC3041', 'blogcatalog3', [0,1], True, use_bias=True)

Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.266714470288
f1_micro:  0.315942591156
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.281414885268
f1_micro:  0.318463925524
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.263938879221
f1_micro:  0.315457719162
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.281339578699
f1_micro:  0.321567106284
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.277281328346
f1_micro:  0.317106283941
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.279887527873
f1_micro:  0.322148952676
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.277448305269
f1_micro:  0.318560899922
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.279637646493
f1_micro:  0.321373157486
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.277812114826
f1_micro:  0.318366951125
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.278660276431
f1_micro:  0.316039565555


In [35]:
for _ in range(10):
    lg('BC3041', 'blogcatalog3', [0,1], True, use_bias=False)

Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.291281802303
f1_micro:  0.326803723817
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.282814555002
f1_micro:  0.323506594259
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.285596599861
f1_micro:  0.320500387898
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.281586769284
f1_micro:  0.322827773468
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.275960608449
f1_micro:  0.323797517455
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.276598472376
f1_micro:  0.322439875873
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.28358502122
f1_micro:  0.321664080683
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.286768835982
f1_micro:  0.319918541505
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.280924907158
f1_micro:  0.323215671063
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.288107729958
f1_micro:  0.323894491854


In [36]:
for _ in range(10):
    lg('BC3041', 'blogcatalog3', [0,1], True, use_bias=False, test_with_training_data=False)

Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.133256165702
f1_micro:  0.233939628483


/home/hoangnt/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.14230853769
f1_micro:  0.240712074303
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.139036480833
f1_micro:  0.241099071207
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.141008272639
f1_micro:  0.245356037152
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.129691463893
f1_micro:  0.233746130031
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.141336083869
f1_micro:  0.242647058824
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.139189968737
f1_micro:  0.238777089783
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.143480778489
f1_micro:  0.248645510836
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.146364133843
f1_micro:  0.249226006192
Experiment  BC3041   blogcatalog3   [0, 1]
f1_macro:  0.14380581783
f1_micro:  0.248645510836


In [37]:
for _ in range(10):
    lg('BC3_deepwalk', 'blogcatalog3', [0,1], True, use_bias=False, test_with_training_data=False)

Experiment  BC3_deepwalk   blogcatalog3   [0, 1]
f1_macro:  0.0785946136219
f1_micro:  0.131191950464
Experiment  BC3_deepwalk   blogcatalog3   [0, 1]
f1_macro:  0.0785162811842
f1_micro:  0.130998452012


/home/hoangnt/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Experiment  BC3_deepwalk   blogcatalog3   [0, 1]
f1_macro:  0.0825238648546
f1_micro:  0.137770897833
Experiment  BC3_deepwalk   blogcatalog3   [0, 1]
f1_macro:  0.0767089389055
f1_micro:  0.129643962848
Experiment  BC3_deepwalk   blogcatalog3   [0, 1]
f1_macro:  0.0827200027725
f1_micro:  0.133900928793
Experiment  BC3_deepwalk   blogcatalog3   [0, 1]
f1_macro:  0.0862535671089
f1_micro:  0.136222910217
Experiment  BC3_deepwalk   blogcatalog3   [0, 1]
f1_macro:  0.0801926892678
f1_micro:  0.132933436533
Experiment  BC3_deepwalk   blogcatalog3   [0, 1]
f1_macro:  0.0783022625852
f1_micro:  0.136029411765
Experiment  BC3_deepwalk   blogcatalog3   [0, 1]
f1_macro:  0.0800083839613
f1_micro:  0.137383900929
Experiment  BC3_deepwalk   blogcatalog3   [0, 1]
f1_macro:  0.0770370373193
f1_micro:  0.130224458204


In [39]:
for _ in range(10):
    lg('BC3042', 'blogcatalog3', [0,1], True, use_bias=False)

Experiment  BC3042   blogcatalog3   [0, 1]
f1_macro:  0.18494642773
f1_micro:  0.209270752521
Experiment  BC3042   blogcatalog3   [0, 1]
f1_macro:  0.182255005803
f1_micro:  0.211307214895
Experiment  BC3042   blogcatalog3   [0, 1]
f1_macro:  0.197762506027
f1_micro:  0.218289371606
Experiment  BC3042   blogcatalog3   [0, 1]
f1_macro:  0.180955690022
f1_micro:  0.206749418154
Experiment  BC3042   blogcatalog3   [0, 1]
f1_macro:  0.192743376177
f1_micro:  0.215186190846
Experiment  BC3042   blogcatalog3   [0, 1]
f1_macro:  0.188248008297
f1_micro:  0.216252909232
Experiment  BC3042   blogcatalog3   [0, 1]
f1_macro:  0.190847697858
f1_micro:  0.215186190846
Experiment  BC3042   blogcatalog3   [0, 1]
f1_macro:  0.184282198754
f1_micro:  0.211307214895
Experiment  BC3042   blogcatalog3   [0, 1]
f1_macro:  0.183350649275
f1_micro:  0.211210240497
Experiment  BC3042   blogcatalog3   [0, 1]
f1_macro:  0.186879310792
f1_micro:  0.216252909232


In [40]:
for _ in range(10):
    lg('BC3043', 'blogcatalog3', [0,1], True, use_bias=False)

Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.201681753326
f1_micro:  0.229635376261
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.205114830894
f1_micro:  0.231380915438
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.203727688398
f1_micro:  0.228277734678
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.203414107914
f1_micro:  0.232738557021
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.197616955301
f1_micro:  0.227695888285
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.194468637217
f1_micro:  0.230314197052
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.20595131003
f1_micro:  0.232156710628
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.196334233198
f1_micro:  0.22730799069
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.201075479853
f1_micro:  0.230993017843
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.201547829383
f1_micro:  0.228956555469


In [41]:
for _ in range(40):
    lg('BC3043', 'blogcatalog3', [0,1], True, use_bias=False)

Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.277342009382
f1_micro:  0.323215671063
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.284405784353
f1_micro:  0.33126454616
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.279097120765
f1_micro:  0.329809930178
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.292801956802
f1_micro:  0.333010085337
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.278391644443
f1_micro:  0.329034134988
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.279402676768
f1_micro:  0.328452288596
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.285421323485
f1_micro:  0.327676493406
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.280475661861
f1_micro:  0.327773467804
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.292934634211
f1_micro:  0.333688906129
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro:  0.284869102815
f1_micro:  0.331555469356
Experiment  BC3043   blogcatalog3   [0, 1]
f1_macro

In [42]:
for _ in range(40):
    lg('BC3044', 'blogcatalog3', [0,1], True, use_bias=False)

Experiment  BC3044   blogcatalog3   [0, 1]
f1_macro:  0.29878070681
f1_micro:  0.342513576416
Experiment  BC3044   blogcatalog3   [0, 1]
f1_macro:  0.304418008242
f1_micro:  0.348622963538
Experiment  BC3044   blogcatalog3   [0, 1]
f1_macro:  0.296591533947
f1_micro:  0.342028704422
Experiment  BC3044   blogcatalog3   [0, 1]
f1_macro:  0.296905232495
f1_micro:  0.345131885182
Experiment  BC3044   blogcatalog3   [0, 1]
f1_macro:  0.311869840782
f1_micro:  0.351241272304
Experiment  BC3044   blogcatalog3   [0, 1]
f1_macro:  0.296035480799
f1_micro:  0.343192397207
Experiment  BC3044   blogcatalog3   [0, 1]
f1_macro:  0.303796589184
f1_micro:  0.34610162917
Experiment  BC3044   blogcatalog3   [0, 1]
f1_macro:  0.293308593938
f1_micro:  0.342804499612
Experiment  BC3044   blogcatalog3   [0, 1]
f1_macro:  0.300354864748
f1_micro:  0.344743987587
Experiment  BC3044   blogcatalog3   [0, 1]
f1_macro:  0.296279546254
f1_micro:  0.342610550815
Experiment  BC3044   blogcatalog3   [0, 1]
f1_macro: